In [15]:
# version 5 project
# pypy/project/Login_using_DB-V5.ipynb
# Using log in form and create db file also, --------------- ok 
# Registration option included. ---------------------------- ok 
# Password Change option added ----------------------------- ok 
# show logged in User name in window ----------------------- ok
# change Password window arranged. ------------------------- ok
# all program close if logout clicked ---------------------- ok

import sqlite3
import hashlib
import tkinter as tk
from tkinter import messagebox

# Create a connection to the SQLite database
conn = sqlite3.connect('user.db')
c = conn.cursor()

# Create a table to store user information
c.execute('''CREATE TABLE IF NOT EXISTS users
             (username TEXT PRIMARY KEY, password TEXT)''')
conn.commit()

# Function to register a new user
def register():
    username = username_entry.get()
    password = password_entry.get()
    
    # Check if the username already exists
    c.execute("SELECT * FROM users WHERE username=?", (username,))
    if c.fetchone():
        messagebox.showerror("Error", "Username already exists. Please choose a different one.")
    else:
        # Hash the password before storing it
        hashed_password = hashlib.sha256(password.encode()).hexdigest()
        c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, hashed_password))
        conn.commit()
        messagebox.showinfo("Success", "Registration successful.")

# ----------------------------------------------------- ok start--------------------
# # Function to log in a user
# def login():
#     username = username_entry.get()
#     password = password_entry.get()
    
#     # Hash the provided password
#     hashed_password = hashlib.sha256(password.encode()).hexdigest()
#     # Check if the username and password match
#     c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, hashed_password))
#     if c.fetchone():
#         messagebox.showinfo("Success", "Login successful.")
#         show_logged_in_menu(username)
#         window.withdraw()  # Hide the login window after successful login
#     else:
#         messagebox.showerror("Error", "Invalid username or password.")


# ---------------------------------------------------------
# Define global variables to store the credentials
logged_in_username = None
logged_in_password = None

# Modify the login function to store credentials after successful authentication
def login():
    global logged_in_username, logged_in_password
    username = username_entry.get()
    password = password_entry.get()
    
    # Hash the provided password
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    # Check if the username and password match
    c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, hashed_password))
    if c.fetchone():
        messagebox.showinfo("Success", "Login successful.")
        # Store the credentials
        logged_in_username = username
        logged_in_password = password
        show_logged_in_menu(username)
        window.withdraw()  # Hide the login window after successful login
    else:
        messagebox.showerror("Error", "Invalid username or password.")

# Function to reuse stored credentials for another website or software
def reuse_credentials():
    global logged_in_username, logged_in_password
    if logged_in_username and logged_in_password:
        # Use the stored credentials to log in to another website or software
        # Replace this with your automation code to log in to the other software
        print("Logging in with stored credentials:")
        print("Username:", logged_in_username)
        print("Password:", logged_in_password)
    else:
        print("No stored credentials found. Please log in first.")




# Example usage:
# You can call the `reuse_credentials()` function when you want to reuse the stored credentials for logging in elsewhere.

#  You may want to add a button or a menu option in your UI to trigger the reuse_credentials() function.

# --------------



#  ---------------------------------------------------------

def show_logged_in_menu(username):
    def logout():
        logged_in_window.destroy()
        window.destroy()  # Close the main application window upon logout
        # exit()  # Exit the program entirely

    def on_closing():
        window.destroy()
        # exit()

    logged_in_window = tk.Toplevel(window)
    logged_in_window.title("Logged In Menu")
    
    username_label = tk.Label(logged_in_window, text="Welcome, " + username)
    username_label.pack(anchor="e", padx=10, pady=5)
    
    logout_button = tk.Button(logged_in_window, text="Logout", command=logout)
    logout_button.pack(padx=10, pady=5)
    
    change_password_button = tk.Button(logged_in_window, text="Change Password", command=lambda: show_change_password_window(username))
    change_password_button.pack(padx=10, pady=5)
    
    logged_in_window.protocol("WM_DELETE_WINDOW", on_closing)




# ----------------------------------------------------

# Function to show the change password window

def show_change_password_window(username):
    change_password_window = tk.Toplevel(window)
    change_password_window.title("Change Password")
    
    old_password_label = tk.Label(change_password_window, text="Old Password:")
    old_password_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
    old_password_entry = tk.Entry(change_password_window, show="*")
    old_password_entry.grid(row=0, column=1, padx=10, pady=5)

    new_password_label = tk.Label(change_password_window, text="New Password:")
    new_password_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
    new_password_entry = tk.Entry(change_password_window, show="*")
    new_password_entry.grid(row=1, column=1, padx=10, pady=5)

    change_button = tk.Button(change_password_window, text="Change Password", command=lambda: change_password(username, old_password_entry.get(), new_password_entry.get()))
    change_button.grid(row=2, columnspan=2, padx=10, pady=5)

# Function to change password
def change_password(username, old_password, new_password):
    # Hash the old password
    hashed_old_password = hashlib.sha256(old_password.encode()).hexdigest()
    
    # Check if the provided old password is correct
    c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, hashed_old_password))
    if c.fetchone():
        # Hash the new password before storing it
        hashed_new_password = hashlib.sha256(new_password.encode()).hexdigest()
        c.execute("UPDATE users SET password=? WHERE username=?", (hashed_new_password, username))
        conn.commit()
        messagebox.showinfo("Success", "Password changed successfully.")
    else:
        messagebox.showerror("Error", "Invalid old password.")

# Create a tkinter window
window = tk.Tk()
window.title("Login System")

# Username label and entry
username_label = tk.Label(window, text="Username:")
username_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)
username_entry = tk.Entry(window)
username_entry.grid(row=0, column=1, padx=10, pady=5)

# Password label and entry
password_label = tk.Label(window, text="Password:")
password_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)
password_entry = tk.Entry(window, show="*")
password_entry.grid(row=1, column=1, padx=10, pady=5)

# Register and login buttons
register_button = tk.Button(window, text="Register", command=register)
register_button.grid(row=2, column=0, padx=10, pady=5)
login_button = tk.Button(window, text="Login", command=login)
login_button.grid(row=2, column=1, padx=10, pady=5)




# Start the tkinter event loop
window.mainloop()
reuse_credentials()

# Close the connection to the database
conn.close()


Logging in with stored credentials:
Username: abc
Password: 12
